In [1]:
import random
from src.reader import Reader
import os
import cv2
from itertools import cycle
import pickle

In [2]:
raster_width = 5
raster_height = 3
sign_size = 64
sample_number = 1
number_of_images = 10000

In [3]:
folder = r"../ML-Lab_data/bg_images"
bg_images = ["herbst-baeume-weg.jpg", "sturm-kueste.jpg"]

In [4]:
all_samples = [(a, b) for a in range(raster_width) for b in range(raster_height)]

In [5]:
df = Reader().read_data_to_df(folders=43, pre_path=r"../ML-Lab_data/data/")

C:\Users\holz_so\Documents\Uni\MaschineLearning\verkehrsschilder\ML-Lab_project\src\reader.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(df_tmp)
C:\Users\holz_so\Documents\Uni\MaschineLearning\verkehrsschilder\ML-Lab_project\src\reader.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(df_tmp)
C:\Users\holz_so\Documents\Uni\MaschineLearning\verkehrsschilder\ML-Lab_project\src\reader.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(df_tmp)
C:\Users\holz_so\Documents\Uni\MaschineLearning\verkehrsschilder\ML-Lab_project\src\reader.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

C:\Users\holz_so\Documents\Uni\MaschineLearning\verkehrsschilder\ML-Lab_project\src\reader.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(df_tmp)
C:\Users\holz_so\Documents\Uni\MaschineLearning\verkehrsschilder\ML-Lab_project\src\reader.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(df_tmp)
C:\Users\holz_so\Documents\Uni\MaschineLearning\verkehrsschilder\ML-Lab_project\src\reader.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(df_tmp)
C:\Users\holz_so\Documents\Uni\MaschineLearning\verkehrsschilder\ML-Lab_project\src\reader.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

In [6]:
def get_random_signs():
    signs = []
    for _, row in df.sample(n=sample_number).iterrows():
        img = Reader().read_preprocess_img_without_bounding(row["Folder"]+row["Filename"],(sign_size,sign_size), clahe=False, resize=False)
        h, w = img.shape[:2]
        img = cv2.resize(img, (sign_size,sign_size))
        x1 = row["Roi.X1"]/w
        y1 = row["Roi.Y1"]/h
        x2 = row["Roi.X2"]/w
        y2 = row["Roi.Y2"]/h
        signs.append((img, int(row["ClassId"]), (x1,y1,x2,y2)))
    return signs

In [7]:
def get_bg_images():
    images = []
    for img_name in bg_images:
        images.append(Reader().read_preprocess_img_without_bounding(os.path.join(folder,img_name), (raster_width*sign_size, raster_height*sign_size), clahe=False))
    return images

In [8]:
labels = []
data = []

bg_images = get_bg_images()

for raw_img, i in zip(cycle(bg_images), range(number_of_images)):
    img = raw_img.copy()
    signs = get_random_signs()
    samples = random.sample(all_samples, sample_number)
    label = dict()
    for sign, s in zip(signs, samples):
        sign_img, sign_label, sign_bounding = sign
        img[sign_size*s[1]:sign_size*(s[1]+1), sign_size*s[0]:sign_size*(s[0]+1)] = sign_img
        key = (sign_size*(s[0]+sign_bounding[0]), sign_size*(s[1]+sign_bounding[1]), 
               sign_size*(s[0]+sign_bounding[2]), sign_size*(s[1]+sign_bounding[3]))
        label[key] = sign_label
    labels.append(label)
    data.append(img)
    
    #w, h = raster_width*sign_size, raster_height*sign_size
    #print(label)
    #bounding = next(iter(label))
    #bounding = tuple(map(int, bounding))
    #print(bounding)
    #cv2.rectangle(img, (bounding[0], bounding[1]), (bounding[2], bounding[3]), (0, 0, 255), 1)
    #cv2.namedWindow("image"+str(i), cv2.WINDOW_NORMAL)
    #cv2.imshow("image"+str(i), img)
    #cv2.waitKey()
    
with open(r"../ML-Lab_data/pickle/data_1.pickle", 'wb') as handle:
    pickle.dump(data, handle)
with open(r"../ML-Lab_data/pickle/labels_1.pickle", 'wb') as handle:
    pickle.dump(labels, handle)

print(labels)
cv2.waitKey()
cv2.destroyAllWindows()

KeyboardInterrupt: 